### Preprocessing

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk

original_data_train = pd.read_csv(
    "adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "OrigEthn", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")

original_data_test = pd.read_csv(
    "adult.test",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "OrigEthn", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")


original_data = pd.concat([original_data_test,original_data_train])
original_data.reset_index(inplace = True, drop = True)

original_data.tail()



,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,OrigEthn,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
48838,27,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
48839,40,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
48840,58,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
48841,22,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K
48842,52,Self-emp-inc,287927.0,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024.0,0.0,40.0,United-States,>50K


In [104]:
data=original_data.copy()


data['Child'] = np.where(data['Relationship']=='Own-child', 'ChildYes', 'ChildNo')
data['OrigEthn'] = np.where(data['OrigEthn']=='White', 'CaucYes', 'CaucNo')

data=data.drop(columns=['fnlwgt','Relationship','Country','Education'])

data=data.replace('<=50K.','<=50K')
data=data.replace('>50K.','>50K')

data.tail()


,Age,Workclass,Education-Num,Martial Status,Occupation,OrigEthn,Sex,Capital Gain,Capital Loss,Hours per week,Target,Child
48838,27,Private,12.0,Married-civ-spouse,Tech-support,CaucYes,Female,0.0,0.0,38.0,<=50K,ChildNo
48839,40,Private,9.0,Married-civ-spouse,Machine-op-inspct,CaucYes,Male,0.0,0.0,40.0,>50K,ChildNo
48840,58,Private,9.0,Widowed,Adm-clerical,CaucYes,Female,0.0,0.0,40.0,<=50K,ChildNo
48841,22,Private,9.0,Never-married,Adm-clerical,CaucYes,Male,0.0,0.0,20.0,<=50K,ChildYes
48842,52,Self-emp-inc,9.0,Married-civ-spouse,Exec-managerial,CaucYes,Female,15024.0,0.0,40.0,>50K,ChildNo


In [105]:
data_ohe=data.copy()

data_ohe['Target'] = np.where(data_ohe['Target']=='>50K', 1., 0.)
print(' -> In column Target: label >50K gets 1.')

data_ohe['OrigEthn'] = np.where(data_ohe['OrigEthn']=='CaucYes', 1., 0.)
print(' -> In column '+str('OrigEthn')+': label '+str('CaucYes')+' gets 1.')

data_ohe['Sex'] = np.where(data_ohe['Sex']=='Male', 1., 0.)
print(' -> In column '+str('Sex')+': label '+str('Male')+' gets 1.')

for col in ['Workclass', 'Martial Status', 'Occupation', 'Child']:
    if len(set(list(data_ohe[col])))==2:
        LabelThatGets1=data_ohe[col][0]
        data_ohe[col] = np.where(data_ohe[col]==LabelThatGets1, 1., 0.)
        print(' -> In column '+str(col)+': label '+str(LabelThatGets1)+' gets 1.')
    else:
        print(' -> In column '+str(col)+': one-hot encoding conversion with labels '+str(set(list(data_ohe[col]))))
        data_ohe=pd.get_dummies(data_ohe,prefix=[col],columns=[col])

data_ohe.tail()

 -> In column Target: label >50K gets 1.
 -> In column OrigEthn: label CaucYes gets 1.
 -> In column Sex: label Male gets 1.
 -> In column Workclass: one-hot encoding conversion with labels {'State-gov', 'Self-emp-not-inc', None, 'Federal-gov', 'Never-worked', 'Without-pay', 'Local-gov', nan, 'Self-emp-inc', 'Private'}
 -> In column Martial Status: one-hot encoding conversion with labels {'Married-spouse-absent', 'Married-AF-spouse', None, 'Divorced', 'Separated', 'Widowed', 'Married-civ-spouse', 'Never-married'}
 -> In column Occupation: one-hot encoding conversion with labels {'Exec-managerial', nan, 'Machine-op-inspct', 'Prof-specialty', 'Priv-house-serv', 'Tech-support', 'Adm-clerical', None, 'Armed-Forces', 'Protective-serv', 'Sales', 'Transport-moving', 'Handlers-cleaners', 'Farming-fishing', 'Craft-repair', 'Other-service'}
 -> In column Child: label ChildNo gets 1.


,Age,Education-Num,OrigEthn,Sex,Capital Gain,Capital Loss,Hours per week,Target,Child,Workclass_Federal-gov,...,Occupation_Farming-fishing,Occupation_Handlers-cleaners,Occupation_Machine-op-inspct,Occupation_Other-service,Occupation_Priv-house-serv,Occupation_Prof-specialty,Occupation_Protective-serv,Occupation_Sales,Occupation_Tech-support,Occupation_Transport-moving
48838,27,12.0,1.0,0.0,0.0,0.0,38.0,0.0,1.0,0,...,0,0,0,0,0,0,0,0,1,0
48839,40,9.0,1.0,1.0,0.0,0.0,40.0,1.0,1.0,0,...,0,0,1,0,0,0,0,0,0,0
48840,58,9.0,1.0,0.0,0.0,0.0,40.0,0.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
48841,22,9.0,1.0,1.0,0.0,0.0,20.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
48842,52,9.0,1.0,0.0,15024.0,0.0,40.0,1.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
data_ohe=data_ohe.iloc[1:]

Finally extract the input and output (X and y) matrices as np.arrays for further analyses using sklearn, and split them into a learning and test sample. 

In [107]:
#extract the X and y np.arrays
y=data_ohe['Target'].values.reshape(-1,1)

data_ohe_wo_target=data_ohe.drop(columns=['Target'])

X_col_names=list(data_ohe_wo_target.columns)
X=data_ohe_wo_target.values


#split the learning and test samples
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

#print the np.array shapes 
print('n_train=',X_train.shape[0])
print('n_test=',X_test.shape[0])
print('p=',X_test.shape[1])

#center-reduce the arrays X_train and X_test to make sure all variables have the same scale
X_train_NoScaling=X_train.copy()
X_train=sk.preprocessing.scale(X_train)
X_test_NoScaling=X_test.copy()
X_test=sk.preprocessing.scale(X_test)

n_train= 32724
n_test= 16118
p= 37



### Classification with decision tree

We now train a <i>Decsion tree</i> on the data, which has the interest to be straightforwardly interpretable.


In [108]:
from sklearn.tree import DecisionTreeClassifier

clf_DT=DecisionTreeClassifier(max_depth=5)
clf_DT.fit(X_train,y_train.ravel())

y_test_pred_DT = clf_DT.predict(X_test)

#*** Uncomment the three raws below to see the decision rules ***
#from sklearn import tree
#dot_data = tree.export_graphviz(DTC_clf)
#print(dot_data)

Let's see now the prediction accuracy

This is the best average result so far. Remark however that altough the prediction are clearly the best for the obesrvations with y=0, they are clearly poor for y=1, which contains about 24&percnt; of the observations. There are even more false positive than true positive predictions.

### Classfication with positive discrimination

In [109]:
import random
def discriminate(X,y, p):
    n=len(y.ravel())
    nb_changes=int(n*p)
    # Randomly select and change the values
    indices_to_change = random.sample(range(n), nb_changes)
    z=y
    for idx in indices_to_change:
        if X[idx][X_col_names.index('Sex')]<0 and z[idx]==0.0:
            z[idx] = 1 - z[idx]
    return z



In [128]:
# Compute DI, where the variable contains the values of the variable studied 
#(e.g. sex) prediction is a vector that contains the predicted values of the target
def compute_DI(variable, prediction):
    n=len(variable)
    n10=len([i for i in range(n) if ((variable[i]>0.) & (prediction[i]==0.))])
    n11=len([i for i in range(n) if ((variable[i]>0.) & (prediction[i]==1.))])
    n01=len([i for i in range(n) if ((variable[i]<=0.) & (prediction[i]==1.))])
    n00=len([i for i in range(n) if ((variable[i]<=0.) & (prediction[i]==0.))])
    try:
        return (n01/(n00+n01))+(n11/(n10+n11))
    except: print(n01, n11, n00,n10)
    

In [197]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier


S=X_train_NoScaling[:,X_col_names.index('Sex')].ravel()
print(len(S), len(y_train.ravel()))
known_DI=compute_DI(S, y_train)
# known_DI=0.35 manually adjust if needed 
print('Known DI: ', known_DI)


# Define the number of splits (k) for cross-validation
num_splits = 10
disc_rate=0.0 #1 for full discrimination, 0 for no discrimination 

p=0
C=0.3
clf_disc=DecisionTreeClassifier(max_depth=5)

S=X_test_NoScaling[:,X_col_names.index('Sex')].ravel()




# Create a KFold object to split the data
kf = KFold(n_splits=num_splits, shuffle=True, random_state=41)

# Split your data and train the model on each split
for train_index, test_index in kf.split(X_train):
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train.ravel()[train_index], y_train.ravel()[test_index]


    # Train the model on the current fold
    clf_disc.fit(X_train_fold, discriminate(X_train_fold, y_train_fold,p))
    y_test_pred_fold = clf_disc.predict(X_test_fold)
    acc=accuracy_score(y_test_fold.ravel(),y_test_pred_fold.ravel())
    tpr=np.sum((y_test_pred_fold.ravel()==1)*(y_test_fold.ravel()==1)) / np.sum(y_test_pred_fold.ravel()==1)
    tnr=np.sum((y_test_pred_fold.ravel()==0)*(y_test_fold.ravel()==0)) / np.sum(y_test_pred_fold.ravel()==0)

    cm = metrics.confusion_matrix(y_test_fold.ravel(),y_test_pred_fold.ravel(),labels=[0,1])

    # print("\nAccuracy =",acc)
    # print("True positive rate =",tpr)  #Rem: Equivalent to metrics.precision_score(y_test.ravel(), y_test_pred.ravel(), pos_label=1.)
    # print("True negative rate =",tnr)
    # print('\nConfusion matrix =')
    # print(cm)

    S=X_test_fold[:,X_col_names.index('Sex')].ravel()
    DI=compute_DI(S, y_test_pred_fold.ravel())
    if (known_DI-DI)>0:
        if  p+(known_DI-DI)*C<1:
            p=p+(known_DI-DI)*C
        else:
            p=p
    else:
        p=max(p+(known_DI-DI)*C,0)
    print('p:', p)
    print('DI:', DI)

    


32724 32724
Known DI:  0.4120271596880345
p: 0.0402487758546348
DI: 0.2778645735059185
p: 0.08229157519117947
DI: 0.2718844952328856
p: 0.1199465810673831
DI: 0.28651047343402236
p: 0.1605867946663657
DI: 0.27655978102475914
p: 0.19722685069368567
DI: 0.2898936395969679
p: 0.24093500610065857
DI: 0.2663333083314582
p: 0.2860235337695934
DI: 0.2617320674582517
p: 0.32417917863810314
DI: 0.28484167679300204
p: 0.3427184688026036
DI: 0.35022952580636624
p: 0.3669099454992882
DI: 0.3313889040324193


In [199]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

#### Results for DT
acc_DT=accuracy_score(y_test.ravel(),y_test_pred_DT.ravel())
tpr_DT=np.sum((y_test_pred_DT.ravel()==1)*(y_test.ravel()==1)) / np.sum(y_test_pred_DT.ravel()==1)
tnr_DT=np.sum((y_test_pred_DT.ravel()==0)*(y_test.ravel()==0)) / np.sum(y_test_pred_DT.ravel()==0)

cm_DT = metrics.confusion_matrix(y_test.ravel(),y_test_pred_DT.ravel(),labels=[0,1])

print("\nAccuracy for DT =",acc_DT)
print("True positive rate for DT =",tpr_DT)  #Rem: Equivalent to metrics.precision_score(y_test.ravel(), y_test_pred.ravel(), pos_label=1.)
print("True negative rate for DT=",tnr_DT)
print('\nConfusion matrix for DT =')
print(cm_DT)
S=X_test_NoScaling[:,X_col_names.index('Sex')].ravel()

y_test_pred_DT = clf_DT.predict(X_test).ravel()
DI_DT=compute_DI(S, y_test_pred_DT)

print('DI y_test_pred_DT =',DI_DT)


### Result for DT with positive discrimination



y_test_pred_disc= clf_disc.predict(X_test)

acc_disc=accuracy_score(y_test.ravel(),y_test_pred_disc.ravel())
tpr_disc=np.sum((y_test_pred_disc.ravel()==1)*(y_test.ravel()==1)) / np.sum(y_test_pred_disc.ravel()==1)
tnr_disc=np.sum((y_test_pred_disc.ravel()==0)*(y_test.ravel()==0)) / np.sum(y_test_pred_disc.ravel()==0)

cm_disc = metrics.confusion_matrix(y_test.ravel(),y_test_pred_disc.ravel(),labels=[0,1])

print("\nAccuracy with disc=",acc_disc)
print("True positive rate with disc=",tpr_disc)  #Rem: Equivalent to metrics.precision_score(y_test.ravel(), y_test_pred.ravel(), pos_label=1.)
print("True negative rate with disc=",tnr_disc)
print('\nConfusion matrix with disc=')
print(cm_disc)


S=X_test_NoScaling[:,X_col_names.index('Sex')].ravel()

y_test_pred_disc = clf_disc.predict(X_test).ravel()
DI_disc=compute_DI(S, y_test_pred_disc)

print('DI y_test_pred_disc =',DI_disc)


# Update les cm avec les valeurs pour les hommes et femmes
n=len(y_test)
idx_M=[i for i in range(n) if (X_test_NoScaling[i,X_col_names.index('Sex')]>0)]
idx_W=[i for i in range(n) if (X_test_NoScaling[i,X_col_names.index('Sex')]<=0)]


X_test_M=[[X_test[i][j] for j in range(len(X_test[0]))] for i in idx_M]
X_test_W=[[X_test[i][j] for j in range(len(X_test[0]))] for i in idx_W]


y_test_pred_disc_M = clf_disc.predict(X_test_M).ravel()
y_test_pred_disc_W = clf_disc.predict(X_test_W).ravel()

y_test_pred_DT_M = clf_DT.predict(X_test_M).ravel()
y_test_pred_DT_W = clf_DT.predict(X_test_W).ravel()

y_test_M=[y_test[i] for i in idx_M]
y_test_W=[y_test[i] for i in idx_W]




cm_DT_W = metrics.confusion_matrix(y_test_W,y_test_pred_DT_W.ravel(),labels=[0,1])
print('CM_DT_W:')
print(cm_DT_W)

cm_disc_W = metrics.confusion_matrix(y_test_W,y_test_pred_disc_W.ravel(),labels=[0,1])
print('CM_disc_W:')
print(cm_disc_W)


cm_DT_M = metrics.confusion_matrix(y_test_M,y_test_pred_DT_M.ravel(),labels=[0,1])
print('CM_DT_M:')
print(cm_DT_M)

cm_disc_M = metrics.confusion_matrix(y_test_M,y_test_pred_disc_M.ravel(),labels=[0,1])
print('CM_disc_M:')
print(cm_disc_M)



Accuracy for DT = 0.849050750713488
True positive rate for DT = 0.7785079242365675
True negative rate for DT= 0.8625378759884709

Confusion matrix for DT =
[[11671   573]
 [ 1860  2014]]
DI y_test_pred_DT = 0.2677385521574236

Accuracy with disc= 0.8241096910286636
True positive rate with disc= 0.6888404216648492
True negative rate with disc= 0.8519488292062543

Confusion matrix with disc=
[[11388   856]
 [ 1979  1895]]
DI y_test_pred_disc = 0.3250317713702725
CM_DT_W:
[[4684   60]
 [ 355  234]]
CM_disc_W:
[[4368  376]
 [ 227  362]]
CM_DT_M:
[[6987  513]
 [1505 1780]]
CM_disc_M:
[[7020  480]
 [1752 1533]]
